In [ ]:
!pip install sentence-transformers
!pip install tiktoken
!pip install datasets
!pip install faiss-gpu-cu12
!pip install pdfplumber
import torch
import sentence_transformers
import datasets
import pandas
import json

TRUE = True
FALSE = False

FRESH_START = TRUE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.2/581.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 98.5 MB/s eta 0:00:00


In [ ]:
# [GET DATA TO BUILD RAG AROUND]
if FRESH_START:
  !mkdir -p ~/.kaggle
  !mv kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  !kaggle datasets download -d nandr39/bharatiya-nyaya-sanhita-dataset-bns
  !unzip bharatiya-nyaya-sanhita-dataset-bns.zip -d bns_data


Dataset URL: https://www.kaggle.com/datasets/nandr39/bharatiya-nyaya-sanhita-dataset-bns
License(s): CC-BY-SA-4.0
  0% 0.00/83.0k [00:00<?, ?B/s]
100% 83.0k/83.0k [00:00<00:00, 183MB/s]
Archive:  bharatiya-nyaya-sanhita-dataset-bns.zip
  inflating: bns_data/BNS.md         
  inflating: bns_data/bns_sections.csv  


In [ ]:
if FRESH_START:
  #!kaggle datasets download -d adarshsingh0903/legal-dataset-sc-judgments-india-19502024
  #!unzip legal-dataset-sc-judgments-india-19502024.zip -d sc_data
    # IT DOESN'T WORK T-T use downloaded version uploaded on drive instead
  !cp /content/drive/MyDrive/colab_stuff/court_hearings_rag/archive.zip /content/
  !unzip archive.zip -d /content/sc_judgments


'''
TODO :: TODO, TODO, TODO TODO, TODO TODO, TODOOOOOOOOOOOOOOOOOOOOOO, TODO, TODO...
'''

Streaming output truncated to the last 5000 lines.
  inflating: /content/sc_judgments/supreme_court_judgments/2013/Manoj_Manu_Anr_vs_Union_Of_India_Ors_on_12_August_2013_1.PDF  
  inflating: /content/sc_judgments/supreme_court_judgments/2013/Mashyak_Grih_Sahakari_San_Maryadit_vs_Uman_Habib_Dhuka_Ors_on_18_April_2013_1.PDF  
  inflating: /content/sc_judgments/supreme_court_judgments/2013/Master_Mallikarjun_vs_Divnl_Mgr_National_Ins_Co_Ltd_Anr_on_26_August_2013_1.PDF  
  inflating: /content/sc_judgments/supreme_court_judgments/2013/Md_Eqbal_Anr_vs_State_Of_Jharkhand_on_22_July_2013_1.PDF  
  inflating: /content/sc_judgments/supreme_court_judgments/2013/Md_Ishaque_Ors_vs_State_Of_West_Bengal_And_Ors_on_3_May_2013_1.PDF  
  inflating: /content/sc_judgments/supreme_court_judgments/2013/Mehsana_Nagrik_Sahkari_Bank_Ltd_vs_Shreeji_Cab_Co_Ors_Etc_on_12_July_2013_1.PDF  
  inflating: /content/sc_judgments/supreme_court_judgments/2013/Mgb_Gramin_Bank_vs_Chakrawarti_Singh_on_7_August_2013_1.PDF  


'\nTODO :: TODO, TODO, TODO TODO, TODO TODO, TODOOOOOOOOOOOOOOOOOOOOOO, TODO, TODO...\n'

In [ ]:
"""
with pdfplumber.open("/content/sc_judgments/supreme_court_judgments/2020/Abhilasha_vs_Parkash_on_15_September_2020_1.PDF") as f:
  i = 0
  for page in f.pages:
    print(page.extract_text())
    print("\n\n")
    i += 1
    if i == 3:
      break
This confirms at on every page, no matter what year, the first line is the header and last the footer, we
henceforth remove them both. Though with time, minor details change like old ones have JUDGEMENT as heading
some new ones have J U D G E M E N T
"""

Abhilasha vs Parkash on 15 September, 2020
Abhilasha vs Parkash on 15 September, 2020
Equivalent citations: AIR 2020 SUPREME COURT 4355, AIRONLINE 2020 SC
727
Author: Ashok Bhushan
Bench: M.R. Shah, R. Subhash Reddy, Ashok Bhushan
REPORTABLE
IN THE SUPREME COURT OF INDIA
CRIMINAL APPELLATE JURISDICTION
CRIMINAL APPEAL NO. 615 of 2020
(arising out of SLP (Crl.) No.8260/2018)
ABHILASHA ...APPELLANT(S)
VERSUS
PARKASH & ORS. ...RESPONDENT(S)
J U D G M E N T
ASHOK BHUSHAN,J.
Leave granted.
2. This appeal has been filed by the appellant, daughter of respondent Nos. 1 and 2, challenging the
order of the High Court of Punjab and Haryana at Chandigarh dated 16.08.2018 by which order the
High Court dismissed the application under Section 482 Cr.P.C. filed by the appellant praying for
setting aside the order of the Judicial Magistrate First Class, Rewari dated 16.02.2011 as well as the
order dated 17.02.2014 passed by the Additional Sessions Judge, Rewari.
3. The brief facts necessary to be notic

In [ ]:
def strip_headers_footers(pages):
    cleaned_pages = []

    for i, page in enumerate(pages):
        lines = [l for l in page.split("\n") if l.strip()]

        if i == 0:
            # first page: keep as-is
            cleaned_pages.append("\n".join(lines))
        else:
            if len(lines) > 2:
                lines = lines[1:-1]  # drop header & footer
            cleaned_pages.append("\n".join(lines))

    return "\n\n".join(cleaned_pages)

In [ ]:
path_to_json = '/content/apex.json'
all_cases = []

def save_it(pages, file, file2, id):
  temp = {
      'id' : id,
      'source' : "Supreme Court",
      'title' : file2,
      'text' : strip_headers_footers(pages),
      'meta' : {
          'year' : file
          }
  }
  all_cases.append(temp)


In [ ]:
# [OH BOY! WHAT HAVE I GOTTEN MYSELF INTO ?!]
import pdfplumber
import os
ROOT = "/content/sc_judgments/supreme_court_judgments"

i = 1
for file in os.listdir(ROOT):
  # print(file)
  print(f"\ncurrently in year :: {file}\n currently in case :: ", end=' ') # so i don't die waiting
  for file2 in os.listdir(f"{ROOT}/{file}"):
    print(f"{i}", sep=' ', end=' ')
    with pdfplumber.open(f"{ROOT}/{file}/{file2}") as pdf:
      pages = []
      for page in pdf.pages:
        text = page.extract_text()
        if text:
          pages.append(text)
          # print(pages)
          save_it(pages, file, file2, i)
      i += 1

with open(path_to_json, "w") as f:
  json.dump(all_cases, f, ensure_ascii=False)


currently in year :: 1959
 currently in case ::  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 
currently in year :: 1960
 currently in case ::  152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 

Exception ignored in: <function _xla_gc_callback at 0x7d8bf1bbda80>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
dataset_supereme_high_court = datasets.load_dataset("opennyaiorg/InJudgements_dataset", token="")


README.md:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

data/train-00000-of-00002-add4caaf8fbc6a(…):   0%|          | 0.00/150M [00:00<?, ?B/s]

data/train-00001-of-00002-09ac6bd45d6b36(…):   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11970 [00:00<?, ? examples/s]

In [ ]:
# [BEAUTIFY BNS DATASET]
SAVE_PATH1 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns.json"

if FRESH_START:
  bns_raw = pandas.read_csv("/content/bns_data/bns_sections.csv")
  print(bns_raw.head())
  bns = []
  for i, entries in bns_raw.iterrows():
    temp = {
        'id' : i+1,
        'chapter': entries['Chapter'],
        'section': entries['Section'],
        'title': entries['Section _name'],
        'text': entries['Description']
    }
    bns.append(temp)
  print(bns[0])

  with open(SAVE_PATH1, 'w') as f:
    json.dump(bns, f)
else:
  with open(SAVE_PATH1, 'r') as f:
    bns = json.load(f)

   Chapter    Chapter_name Chapter_subtype  Section  \
0        1     Preliminary     Preliminary        1   
1        1     Preliminary     Preliminary        2   
2        1     Preliminary     Preliminary        3   
3        2  Of punishments  Of punishments        4   
4        2  Of punishments  Of punishments        5   

                               Section _name  \
0  Short title, commencement and application   
1                               Definitions.   
2                       General explanations   
3                                Punishments   
4                   Commutation of sentence    

                                         Description  
0  (1) This Act may be called the Bharatiya Nyaya...  
1  In this Sanhita, unless the context otherwise ...  
2  (1) Throughout this Sanhita every definition o...  
3  The punishments to which offenders are liable ...  
4  The appropriate Government may, without the co...  
{'id': 1, 'chapter': 1, 'section': 1, 'title': 'Sh

In [ ]:
# [BEAUTIFY SUP HI COURT DATASET]
SAVE_PATH2 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_courts.json"
# [THE TEXT DATA IS TOO NOISY WITH HTML TAGS]
def clean_html(text):
  import re
  text = re.sub(r"<span[^>]*>.*?</span>", " ", text, flags=re.DOTALL) # REMOVE 'EM HTML
  text = re.sub(r"<[^>]+>", " ", text) # REMOVE 'EM HTML
  text = re.sub(r"[=\-_.]{4,}", " ", text)

  return text

# print(dataset_supereme_high_court)
# print(dataset_supereme_high_court['train'][0])
if FRESH_START:
  high_courts_raw = dataset_supereme_high_court['train']
  high_courts_raw[0]
  print(high_courts_raw[0].keys())
  high_courts = []
  for i, cases in enumerate(high_courts_raw):
    temp = {
        'id' : i+1,
        'court_details' : f"{cases['Court_Type']} | {cases['Case_Type']} | {cases['Court_Name']}",
        'cited_by' : cases['Cited_by'],
        'title': cases['Titles'],
        # 'text' : " ".join(cases["Text"].split()) # too mych filler mate NEXT DAY REALIZATION : without spaces i have no idea where
        # a paragraph or context ends, so i end up making random chunks distributing meaning(s) very badly, keep it for now, remove
        # at chunking step instead
        'text' : clean_html(cases["Text"])
    }
    high_courts.append(temp)

  print(high_courts[0])
  with open(SAVE_PATH2, "w") as f:
    json.dump(high_courts, f)
else:
  with open(SAVE_PATH2, "r") as f:
    high_courts = json.load(f)

dict_keys(['Titles', 'Court_Name', 'Cites', 'Cited_by', 'Doc_url', 'Text', 'Doc_size', 'Case_Type', 'Court_Type', 'Court_Name_Normalized'])
{'id': 1, 'court_details': 'High_Court | Land&Property | Patna High Court - Orders', 'cited_by': 9, 'title': 'Maheshwar Mandal & Anr vs The State Of Bihar & Ors on 24 June, 2014', 'text': 'IN THE HIGH COURT OF JUDICATURE AT PATNA\n                  Civil Writ Jurisdiction Case No.1091 of 2013\n                                        With\n                      Interlocutory Application No. 1572 of 2013\n                                         In\n                  Civil Writ Jurisdiction Case No. 1091 of 2013\n \n1. Maheshwar Mandal S/O Late Bachchi Mandal\n2. Sanjay Mandal S/O Maheshwar Mandal, both resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivision- Forbesganj, District Araria     Petitioners Versus\n1. The State of Bihar\n2. Divisional Commissioner, Purnea\n3. Land Reforms Deputy Collector, Forbesganj, Araria     Resp

In [ ]:
"""
[FURTHER PRETTIFYING OF DATA]
well i see no point in doing this, but apprearently it helps later on figuring what went wrong in
RAG design and extending it in future as well.
"""

new_bns = []
for entries in bns:
  temp = {
      'id' : f"BNS_{entries['id']}",
      'source' : "BNS",
      'title' : entries['title'],
      'text' : entries['text'],
      'meta' : {
          'chapter' : entries['chapter'],
          'section' : entries['section']
          }
  }
  new_bns.append(temp)
bns = new_bns
del new_bns

new_court = []
for entries in high_courts:
  temp = {
      'id' : f"COURTS_{entries['id']}",
      'source' : "Courts",
      'title' : entries['title'],
      'text' : entries['text'],
      'meta' : {
          'cited_by' : entries['cited_by'],
          'court_details' : entries['court_details']
          }
  }
  new_court.append(temp)
court = new_court # yeh i changed name.
del new_court
del high_courts
print(court[0])
print("\n\n")
print(bns[0])

In [ ]:
# well now since i hesitantly unified representation of both(infuture three:: TODO TODO TODO TODO TODOOOOOOOOOOOOO) datasets' embeddings
# i can write a single function to break 'em all up in chunks and save the relevant info only.

def make_chunks(text, tokenizer, token_len=400, overlap_len=60):
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    chunks = []
    current_tokens = []

    for para in paragraphs:
        para_tokens = tokenizer.encode(para)

        # if paragraph alone is too big, split it
        if len(para_tokens) > token_len:
            for i in range(0, len(para_tokens), token_len - overlap_len):
                chunks.append(para_tokens[i:i + token_len])
            current_tokens = []
            continue

        # if adding paragraph exceeds budget → flush
        if len(current_tokens) + len(para_tokens) > token_len:
            chunks.append(current_tokens)
            # overlap
            current_tokens = current_tokens[-overlap_len:] if overlap_len else []

        current_tokens.extend(para_tokens)

    if current_tokens:
        chunks.append(current_tokens)

    return chunks


In [ ]:
'''
hear the news, faiss returns integer index of what was used to embed, so, embedding_list[i]  <same order>  metadata_list[i]
'''
